# Sources

https://github.com/nicknochnack/KerasRL-OpenAI-Atari-SpaceInvadersv0/blob/main/Space%20Invaders%20Walkthrough.ipynb

We followed a similar structure to this source as it gave us a general idea of how to use OpenAI Gym and what libraries worked will with it. Instead of using a pixel state space, we opted to instead use a RAM state space as there were very few implementations with RAM state spaces available. Also, we opted to use very different neural network layers as it did not make sense to use convolutional layers with a RAM state space as the input. 

# 0. Install Dependencies

In [ ]:
!pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [ ]:
import gym 
import random

In [ ]:
# RAM state space environment
env = gym.make('SpaceInvaders-ram-v0')

# height, width, channels = env.observation_space.shape
ram_obs = env.observation_space.shape[0]
actions = env.action_space.n

In [ ]:
# Agent that performs actions randomly for a specified number of episodes

episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [ ]:
# def build_model(height, width, channels, actions):
    
def build_model(ram_obs, actions):
    model = Sequential()
    
    #model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    #model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    #model.add(Convolution2D(64, (3,3), activation='relu'))
    #model.add(Flatten())
    #model.add(Dense(512, activation='relu'))
    #model.add(Dense(256, activation='relu'))
    model.add(Flatten(input_shape=(1,128))),
    model.add(Dense(512, input_dim=ram_obs, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
# run this if encountering errors when trying to build agent below

del model

In [ ]:
# run this if encountering errors when trying to build agent below

del dqn

In [ ]:
# builds model
model = build_model(ram_obs, actions)

In [ ]:
# summary of the neural network of the model
model.summary()

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    
    # balance exploration and exploitation and decay agent 
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=2000000)
    
    # provides a fast and efficient data structure that we can store the agent’s experiences in
    memory = SequentialMemory(limit=1000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [ ]:
# build agent and compile with adam optimizer algorithm for specified learning rate
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=3e-4))

In [ ]:
# train the DQNAgent for specified number of steps
dqn.fit(env, nb_steps=2000000, visualize=False, verbose=3)

In [ ]:
# test the trained agent
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

# 4. Reloading Agent from Memory

In [ ]:
# allows saving weights
dqn.save_weights('weights5.h5f')

In [ ]:
# useful for not having to reload entire notebook
# del model, dqn

In [ ]:
# allows loading weights
dqn.load_weights('weights5.h5f')